In [1]:
# imports

import pandas as pd

## Load Emotions Dataset

In [2]:
emo_train_df = pd.read_csv("./Swahili Emotion Data/emo_train.csv", encoding = "ISO-8859-1")
display(emo_train_df)

emo_valid_df = pd.read_csv("./Swahili Emotion Data/emo_valid.csv", encoding = "ISO-8859-1")
display(emo_valid_df)

emo_test_df = pd.read_csv("./Swahili Emotion Data/emo_test.csv", encoding = "ISO-8859-1")
display(emo_test_df)

,text,labels
0,Ni zaidi ya Asha ngedere,[4]
1,Tuendelee kuchapa kazi ndugu zangu tena kwa fu...,[1]
2,waziri anasoma utadhani uamefungwa mashine,[0]
3,Ukweli ni kwamba mjinga ndie ambae haoni Magu...,"[4, 5]"
4,Hapo mlijiridhisha nn sasa??,[5]
...,...,...
9727,"Tunaomba muongeze siku za mkopo kidogo, wanafu...",[3]
9728,hukupaswa kulimbikiza vyombo vichafu kwenye si...,[4]
9729,"Pole sana wafiwa. Pole mama Janet, pole sana m...",[3]
9730,Ninahisi kuharibiwa na ni mbaya zaidi kwa mama...,[3]


,text,labels
0,Hee makubwa?? wanasema washa zoea mabomu??,[5]
1,"Hakuna anayekataa foreign investors, tatizo hi...",[4]
2,Utazidi kufanikiwa kwa sababu hukuwasahau waza...,[1]
3,Mbaya?,[0]
4,Cha kuchekesha hapa ni kipi sasa! Watu walisha...,[5]
...,...,...
1292,Kunywa maji upunguze presha,[0]
1293,Sio vyema jamani uchungu wa mtoto kisha nakuib...,[3]
1294,Afadhali yako. Ni vitu gani umewekea jitihada ...,[1]
1295,Baadhi ya wanafunzi walishindwa kustahimili ma...,[3]


,text,labels
0,"Sasa walipokuja, walikuja kama nani? wapiga ch...",[5]
1,Waziri wa kilimo na naibu wake wapo vizuri san...,[1]
2,"Akajielekeza, hisia zake za uchungu zikimujaa.",[3]
3,Yakobo alijawa na huzuni kwa yule mwanamke amb...,[3]
4,nimepata maumivu ya kichwa na kikohozi .,[3]
...,...,...
1942,"""Ah ni ya kuchekesha!"" Alisema Hidaya, naye a...",[1]
1943,"Mama yao alikufa mwezi mmoja baadaye, aliumia.",[3]
1944,"Watu walianza kupukutika kwa kasi,, aibu na ha...","[3, 6]"
1945,ni unamaanisha nini?nimenunuwa kompyuta hii mi...,[5]


## Load Pre-Trained Model
### AfriBerta

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("castorini/afriberta_base")
model = AutoModelForMaskedLM.from_pretrained("castorini/afriberta_base")

c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


## Preprocess Data

In [ ]:
import torch
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer(text, 
                                  max_length=self.max_length, 
                                  padding='max_length', 
                                  truncation=True,
                                  return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create the datasets
train_dataset = EmotionDataset(train_df['text'].tolist(), train_df['label_id'].tolist(), tokenizer)
valid_dataset = EmotionDataset(valid_df['text'].tolist(), valid_df['label_id'].tolist(), tokenizer)
test_dataset = EmotionDataset(test_df['text'].tolist(), test_df['label_id'].tolist(), tokenizer)


In [ ]:
# tokenise data and  

